特征金字塔(Feature Pyramid Networks)
===

特征金字塔的概念在很早以前就已经有了，好比目标检测还在使用滑动窗的时候，特征金字塔能够改善目标尺度变化的问题。

![images](images/03_07_001.jpg)

- (a)图是一个图像金字塔，金字塔的构建只靠缩放图像来完成，而后再不一样尺度的图像上分别提取特征，这样一来特征是相互独立的，并且很是耗时，固然它和CNN没有半毛钱关系；
- (b)图是一个常规的卷积操做，金字塔的每一层就再也不是图像了，而是提取获得的特征，而后再最后一层特征上作预测，YOLO和Faster R-CNN都是这样，它只要用到了单层的信息；
- (c)图和(b)图的前向过程是一致的，区别在于预测的时候，拉取的多层的分支，就有了多尺度的概念，好比SSD；
- (d)图的自底向上的过程和以前两个也是一致的，可是除此以外，有加入了一个逆过来的自顶向下的过程，而且两个过程间有横向的交互，最后的预测在第二个过程的特征图上进行，这个结构被做者称为特征金字塔网络。

# 1.SSD中的特征金字塔

SSD首次在卷积网络中金字塔特征层及上进行尝试，类似于特征图像金字塔。由于，SSD会重复利用前向传播过程中得到的多尺寸的feature map，因此，不会消耗太多的计算资源。SSD为了避免低层次的特征，直接在网络的较深层进行金字塔的搭建（比如在VGG 网络中的conv4-3层）并添加几层网络。从而无法得到较前层高分辨率的信息，不利于小目标的检测。SSD没有利用到足够底层的特征，因为底层特征语义信息弱，但是底层特征分辨率高，对检测小目标很重要

# 2.FPN
FPN接受一个具有任意尺寸的单尺度图像作为输入，在多个层级以全卷积的形式生成不同尺寸比例的特征图。FPN的构建涉及三个部分
- 自底向上的路径(Bottom-up pathway)
- 自顶向下的路径(top-down pathway)
- 横向连接(lateral connection)

## 2.1.自底向上
自底向上的路径就是主干网络backbone的前向计算，产生不同尺度的feature map，尺度的比例为2，即每次下采样都是缩小2倍。自底向上的过程，空间分辨率降低，但是语义性增加。这里把那些会输出同样尺寸feature map的层归为一个stage。每个stage都定义了一级金字塔，每个stage的最后一层特征被选取出来。一般使用的是ResNet，选取的是每一个stage最后一个残差块的输出，将Conv2，Conv3，Conv4，Conv5的输出定义为{C2，C3，C4，C5}。这些输出相对于输入图像，stride为{4，8，16，32}，即分辨率缩小的倍数。为啥不用C1呢，因为维度太高了，内存消耗大。

## 2.2.自顶向下和横向连接
FPN通过自顶向下的方式，从语义信息丰富的层出发，构建出分辨率更高的层，将这些层的特征与浅层的特征通过横向连接融合。如下图，对于空间分辨率较粗糙的深层特征，进行2倍的最近邻上采样，相应的浅层特征使用1x1的卷积降维，之后与上采样的特征通过逐元素相加合并。合并后的特征又通过3x3的卷积生成最终的feature map，即{P2，P3，P4，P5}，这一步降低了上采样的混叠效应（aliasing effect）

![images](images/03_07_002.png)

FPN可以更加详细的用下图表示

![images](images/03_07_003.png)

金字塔中所有的层都共享分类器和回归器，就像传统的图像金字塔那样。固定特征的通道数为256，因此所有额外的层输出都为256通道。这些额外的层没有使用非线性。

# 3.自适应特征融合-Adaptively Spatial Feature Fusion(ASFF)

## 3.1.特征金字塔的缺点

特征金字塔的主要缺点之一是不同尺度特征之间的不一致性，尤其对于一阶检测器而言。具体而言，在检测对象时采用启发式引导的特征选择：大目标通常与较高特征图相关联，而小目标通常与较低特征图相关联。一旦一个目标与某一特定层级特征图相关联（正样本），其他层级特征图的相应位置将会被视为背景（负样本）。因此，一旦图像中既含有大目标又含有小目标时，不同层级之间的特征存在冲突（他们都想在特征金字塔中占据主要地位）。这种冲突会干扰训练期间的梯度计算，并降低特征金字塔的有效性。

## 3.2.ASFF的提出

ASFF用来解决一阶检测器中特征金字塔内部的不一致性。ASFF使网络能够直接学习如何在其他级别对特征进行空间滤波，从而仅保留有用的信息以进行组合。对于某个级别的特征，首先将其他级别的特征调整为相同的分辨率并简单集成，然后训练以找到最佳的融合方式。在每个空间位置，将不同级别的特征自适应地融合在一起，例如：若某位置携带矛盾的信息，则这些特征将会被滤除，若某位置的特征带有更多的区分性线索，则这些特征将会被增强。

## 3.3.ASFF的优点

- 由于搜索最优融合的操作是可微的，因此可以方便地利用bp算法学习
- 与基础网络无关，可以应用于所有具有特征金字塔结构的单阶检测器
- 实现简单，附加计算成本很小。

## 3.4.ASFF算法描述

![images](images/03_07_004.jpg)

左边就是一个普通的FPN，绿色框描述了如何将特征进行融合，其中$X^1,X^2,X^3$分别为来自level1，level2，level3这三个层的特征。然后level1，level2，level3这三个层的特征分别乘上权重参数$\alpha^3, \beta^3, \gamma^3$并求和，就可以得到新的融合后的特征ASFF-3。这个过程可以用以下公式来表示：
$$y_{ij}^l=\alpha_{ij}^l \cdot x_{ij}^{1 \to l}+\beta_{ij}^l \cdot x_{ij}^{2 \to l} + \gamma_{ij}^l \cdot x_{xj}^{3 \to l}$$

因此这里是相加的操作，所以需要加的时候必须保证leve1，level2，level3输出的特征相同，且通道数也要相同，所以需要对不同的特征层做上采样或者下采样并调整通道数来满足条件。